In [160]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.models import Title
from math import pi

In [26]:
#Reading in data

pick_drop_data = pd.read_csv('data/pickup_dropoff.csv', dtype={"postal": str})

In [27]:
#Checking types of the data 
pick_drop_data.dtypes

datetime_utc_0          object
actual_time_utc_0       object
postal                  object
booking_id              object
committed_time_utc_0    object
estimated_time_utc_0    object
status                  object
type                    object
dtype: object

In [28]:
#Chosing columns with dates and creating new columns for berlin dates
localize_dates = ["datetime_utc_0", "actual_time_utc_0", "committed_time_utc_0", "estimated_time_utc_0"]
berlin_dates = [date.replace('0', 'berlin') for date in localize_dates]

In [29]:
def berlin_time(df, time_columns_names):
    """
    Given a dataset and time columns converts data from utc 0 to Berlin time.
    df: pandas data frame
    time_columns_names: columns that contain time to be converted.
    
    returns: data frame with columns with time converted.
    """
    #Replace to it can be localize to utc 0
    data_date = df[time_columns_names].apply(lambda x: x.str.replace('\+\d\d:\d\d', ""))
    data_date = data_date.apply(lambda x: pd.to_datetime(x).dt.tz_localize('UTC').dt.tz_convert('Europe/Berlin'))
    return data_date
    
    
    

In [30]:
pick_drop_data[berlin_dates] = berlin_time(pick_drop_data, localize_dates)

In [31]:
pick_drop_data.dtypes

datetime_utc_0                                      object
actual_time_utc_0                                   object
postal                                              object
booking_id                                          object
committed_time_utc_0                                object
estimated_time_utc_0                                object
status                                              object
type                                                object
datetime_utc_berlin          datetime64[ns, Europe/Berlin]
actual_time_utc_berlin       datetime64[ns, Europe/Berlin]
committed_time_utc_berlin    datetime64[ns, Europe/Berlin]
estimated_time_utc_berlin    datetime64[ns, Europe/Berlin]
dtype: object

## Data validation

#### Checking nulls in data set

In [38]:
#Checking if nulls in columns that should be complete
pick_drop_data[["datetime_utc_berlin", 
                "committed_time_utc_berlin", 
                "estimated_time_utc_berlin", 
                "status", "type", "booking_id", "postal"]].notnull().all()

datetime_utc_berlin           True
committed_time_utc_berlin     True
estimated_time_utc_berlin     True
status                        True
type                          True
booking_id                    True
postal                       False
dtype: bool

According to what it is observed, there are certain cases (30 in total) where the postal code is not recorded in the data set. As shown below, these records also include services that were provided (i.e. where transactions where closed).

In [46]:
pick_drop_data[(pick_drop_data["postal"].isnull()) & (pick_drop_data["status"]=="closed")]

,datetime_utc_0,actual_time_utc_0,postal,booking_id,committed_time_utc_0,estimated_time_utc_0,status,type,datetime_utc_berlin,actual_time_utc_berlin,committed_time_utc_berlin,estimated_time_utc_berlin
496,2017-06-09T18:24:21.000+02:00,2017-06-09T18:24:21.000+02:00,NaN,d0bc9130-4b97-40db-a18a-d8ac85e14188,2017-06-09T18:25:47.000+02:00,2017-06-09T18:25:47.000+02:00,closed,pickup,2017-06-09 20:24:21+02:00,2017-06-09 20:24:21+02:00,2017-06-09 20:25:47+02:00,2017-06-09 20:25:47+02:00
2228,2017-06-10T14:44:11.000+02:00,2017-06-10T14:44:11.000+02:00,NaN,dbb7e1e0-e2ae-42c9-9d57-5bea8885b4a7,2017-06-10T14:42:14.000+02:00,2017-06-10T14:43:55.000+02:00,closed,dropoff,2017-06-10 16:44:11+02:00,2017-06-10 16:44:11+02:00,2017-06-10 16:42:14+02:00,2017-06-10 16:43:55+02:00
4519,2017-06-10T11:11:26.000+02:00,2017-06-10T11:11:26.000+02:00,NaN,2935b385-008d-4ba0-97fc-4dda5b49147b,2017-06-10T11:09:39.000+02:00,2017-06-10T11:11:11.000+02:00,closed,dropoff,2017-06-10 13:11:26+02:00,2017-06-10 13:11:26+02:00,2017-06-10 13:09:39+02:00,2017-06-10 13:11:11+02:00
7007,2017-06-10T23:42:15.000+02:00,2017-06-10T23:42:15.000+02:00,NaN,a7d87dda-91e9-456b-8a6a-eb0c2c9f6bdc,2017-06-10T23:37:34.000+02:00,2017-06-10T23:37:34.000+02:00,closed,pickup,2017-06-11 01:42:15+02:00,2017-06-11 01:42:15+02:00,2017-06-11 01:37:34+02:00,2017-06-11 01:37:34+02:00


`actual_time_utc_berlin` should have missing values. This is because we only expect to have a non missing if the service was actually provided. In total we see that we have 2110 actual times. Note, however that we would be expecting all of these actual times to correspond to closed services. This is the case in 2026 cases where pickup and drop off services were provided. In 84 observations these actual times correspond to cancelled pickup services.

In [58]:
pick_drop_data['actual_time_utc_berlin'].notnull().sum()

2110

In [64]:
#Checking number of actual times with complete status
pick_drop_data[(pick_drop_data['actual_time_utc_berlin'].notnull()) 
               & (pick_drop_data['status'] == "closed")].shape[0]

2026

In [65]:
#Checking number of actual times with cancelled status
pick_drop_data[(pick_drop_data['actual_time_utc_berlin'].notnull()) 
               & (pick_drop_data['status'] == "cancelled")].shape[0]

84

#### Checking that number of pick ups coincides with number of drop off

In [241]:
len(pick_drop_data[(pick_drop_data['status'] == 'closed') & (pick_drop_data['type']=='pickup')])

1014

In [242]:
len(pick_drop_data[(pick_drop_data['status'] == 'closed') & (pick_drop_data['type']=='dropoff')])

1012

A simple look up of the number of closed rides with pickup and drop offs shows that there is a small difference between them. In what follows, I will find exactly for which booking id are there a pick up but not a drop off or viceversa.

In [243]:
#Pick ups without drop offs

#Generate list of all the drop off booking id's
book_id_dropoff = list(pick_drop_data[(pick_drop_data['status'] == 'closed') & (pick_drop_data['type']=='dropoff') ]['booking_id'])

#Check which ones are missing
pick_drop_data[(pick_drop_data['status'] == 'closed') & (pick_drop_data['type']=='pickup') 
          & ~(pick_drop_data['booking_id'].isin(book_id_dropoff))]

,datetime_utc_0,actual_time_utc_0,postal,booking_id,committed_time_utc_0,estimated_time_utc_0,status,type,datetime_utc_berlin,actual_time_utc_berlin,committed_time_utc_berlin,estimated_time_utc_berlin
7007,2017-06-10T23:42:15.000+02:00,2017-06-10T23:42:15.000+02:00,NaN,a7d87dda-91e9-456b-8a6a-eb0c2c9f6bdc,2017-06-10T23:37:34.000+02:00,2017-06-10T23:37:34.000+02:00,closed,pickup,2017-06-11 01:42:15+02:00,2017-06-11 01:42:15+02:00,2017-06-11 01:37:34+02:00,2017-06-11 01:37:34+02:00
7033,2017-06-10T23:53:17.000+02:00,2017-06-10T23:53:17.000+02:00,10559,7e199ca8-b04a-495d-bfd6-edd1984b415f,2017-06-10T23:51:24.000+02:00,2017-06-10T23:51:24.000+02:00,closed,pickup,2017-06-11 01:53:17+02:00,2017-06-11 01:53:17+02:00,2017-06-11 01:51:24+02:00,2017-06-11 01:51:24+02:00
8642,2017-06-10T23:48:05.000+02:00,2017-06-10T23:48:05.000+02:00,10969,183a2ba6-6c0f-4059-91f2-165e52c89ffb,2017-06-10T23:48:03.000+02:00,2017-06-10T23:48:09.000+02:00,closed,pickup,2017-06-11 01:48:05+02:00,2017-06-11 01:48:05+02:00,2017-06-11 01:48:03+02:00,2017-06-11 01:48:09+02:00
8650,2017-06-10T23:51:46.000+02:00,2017-06-10T23:51:46.000+02:00,10117,723130ac-08ce-4838-9def-1c0e49763f0f,2017-06-10T23:54:37.000+02:00,2017-06-10T23:54:00.000+02:00,closed,pickup,2017-06-11 01:51:46+02:00,2017-06-11 01:51:46+02:00,2017-06-11 01:54:37+02:00,2017-06-11 01:54:00+02:00
8658,2017-06-10T23:53:48.000+02:00,2017-06-10T23:53:48.000+02:00,12045,d755fc90-555c-44bb-96eb-4eec23e8297b,2017-06-10T23:52:32.000+02:00,2017-06-10T23:52:32.000+02:00,closed,pickup,2017-06-11 01:53:48+02:00,2017-06-11 01:53:48+02:00,2017-06-11 01:52:32+02:00,2017-06-11 01:52:32+02:00


In [244]:
#Drop offs without pickups

#Generate list of all the drop off booking id's
book_id_pickup = list(pick_drop_data[(pick_drop_data['status'] == 'closed') & (pick_drop_data['type']=='pickup') ]['booking_id'])

#Check which ones are missing
pick_drop_data[(pick_drop_data['status'] == 'closed') & (pick_drop_data['type']=='dropoff') 
          & ~(pick_drop_data['booking_id'].isin(book_id_pickup))]

,datetime_utc_0,actual_time_utc_0,postal,booking_id,committed_time_utc_0,estimated_time_utc_0,status,type,datetime_utc_berlin,actual_time_utc_berlin,committed_time_utc_berlin,estimated_time_utc_berlin
7057,2017-06-11T00:14:08.000+02:00,2017-06-11T00:14:08.000+02:00,10243,efdf2025-84d6-4336-a0af-c853156694e3,2017-06-11T00:15:12.000+02:00,2017-06-11T00:15:12.000+02:00,closed,dropoff,2017-06-11 02:14:08+02:00,2017-06-11 02:14:08+02:00,2017-06-11 02:15:12+02:00,2017-06-11 02:15:12+02:00
8679,2017-06-11T00:15:26.000+02:00,2017-06-11T00:15:26.000+02:00,10967,bad30cdb-0434-43ff-be0f-c1f4bd7143d4,2017-06-11T00:23:09.000+02:00,2017-06-11T00:23:09.000+02:00,closed,dropoff,2017-06-11 02:15:26+02:00,2017-06-11 02:15:26+02:00,2017-06-11 02:23:09+02:00,2017-06-11 02:23:09+02:00
8681,2017-06-11T00:19:03.000+02:00,2017-06-11T00:19:03.000+02:00,10437,9899b627-c082-4945-a549-d4119792c9c6,2017-06-11T00:19:07.000+02:00,2017-06-11T00:19:07.000+02:00,closed,dropoff,2017-06-11 02:19:03+02:00,2017-06-11 02:19:03+02:00,2017-06-11 02:19:07+02:00,2017-06-11 02:19:07+02:00


## Analizing the demand

A good way to determine the demand of the services is to check the number of times the users requested a service. We can check this per day and hour.

Note that a first problem we have to deal with when grouping is when to count a request. Allygator shuttle services runs through the night, till the other day (2 a.m). As in the data sets we have observations that are repeated, we might be overestimating the demand if we count a request double. 

For example, we will be counting double if we count as a separate request one made at 23:59 on 09.06.17 and the observation from the same requests that continues being open at 00:01 10.06.17. As a rule, we will say that this request belongs to the day in which it was made 09.06.17.

For this purpose, to measure the number of requests we will take the first observation of the request. 


In [274]:
first_request = pick_drop_data.sort_values(by=["datetime_utc_berlin", "type"], 
                                           ascending=[True, False]).groupby('booking_id').agg(lambda x: x.iloc[0]).reset_index()

In [277]:
first_request.columns

Index(['booking_id', 'datetime_utc_0', 'actual_time_utc_0', 'postal',
       'committed_time_utc_0', 'estimated_time_utc_0', 'status', 'type',
       'datetime_utc_berlin', 'actual_time_utc_berlin',
       'committed_time_utc_berlin', 'estimated_time_utc_berlin'],
      dtype='object')

In [275]:
def group_per_date(df, grouping_period, date_col, agg_col, stat):
    """
    df: pandas data frame.
    grouping_pediod: (str) a date period grouping e(e.g. 1h: hourly, 1d:daily).
    date_col: (str) column in date set with date time object.
    agg_key: (str) column to be aggregated on
    stat: (str) numpy operation (np.sum) this is the type of aggregation to be performed.
    
    return: data frame grouped in given period
    """
    
    return df.groupby(pd.Grouper(freq=grouping_period, key=date_col)).agg({agg_col:stat}).reset_index()




In [288]:
request_per_day = group_per_date(first_request, "1d", "datetime_utc_berlin", "booking_id", "count")

In [289]:
#Getting labels for the graph
request_per_day["date"] = request_per_day['datetime_utc_berlin'].apply(lambda x: x.strftime('%a %b %Y'))

In [290]:
p = figure(plot_width=400, plot_height=400, x_range=list(request_per_day["date"]),
          x_axis_label="Date", y_axis_label="Count")
p.vbar(x=request_per_day["date"], width=0.4, top=request_per_day['booking_id'])
p.add_layout(Title(text="Number of requests per day", align="center", text_font_size="12pt"), "above")
show(p)

In [295]:
requests_day_hour = group_per_date(first_request, "1h", "datetime_utc_berlin", "booking_id", "count")

In [296]:
#Excluding 0's autogenerated
requests_day_hour = requests_day_hour[~(requests_day_hour['booking_id'] == 0)]
requests_day_hour["date_hour"] = requests_day_hour['datetime_utc_berlin'].apply(lambda x: x.strftime('%H H %a'))

In [297]:
#Plotting the number of requests within an hour / day
line_fig = figure(plot_width=600, plot_height=600, x_range=list(requests_day_hour["date_hour"]),
          x_axis_label="Date", y_axis_label="Count")
line_fig.line(requests_day_hour["date_hour"], requests_day_hour["booking_id"])
line_fig.circle(requests_day_hour["date_hour"], requests_day_hour["booking_id"])
line_fig.add_layout(Title(text="Number of requests within hour day", align="center", text_font_size="12pt"), "above")
line_fig.xaxis.major_label_orientation = pi/4
show(line_fig)

### Analyzing satisfied demand - rides

Not all the requests of allygator are completed. Sometimes because of different circumstances (e.g. waiting times) users cancel their requests. Here I want to understand how many of the requests previously studied are actually served by the shuttle service.

** Note that I can also use the other data set to obtain the total number of passengers that use the ride. **

In [265]:
pick_up_closed = pick_drop_data[(pick_drop_data['status'] == 'closed') & (pick_drop_data['type']=='pickup')]
drop_closed = pick_drop_data[(pick_drop_data['status'] == 'closed') & (pick_drop_data['type']=='drop')]

In [258]:
request_pickup_closed = pd.merge(first_request, pick_up_closed, how="outer", 
         indicator=True, on="booking_id",
         suffixes = ["_req", "_pick_closed"])

In [285]:
pick_up_per_day = group_per_date(request_pickup_closed[request_pickup_closed["_merge"]=="both"], "1d", "datetime_utc_berlin_req", "booking_id", "count")

In [293]:
p.vbar(x=request_per_day["date"], width=0.4, top=pick_up_per_day['booking_id'], color="firebrick")
show(p)